In [12]:
import lightgbm as lgb
import pandas as pd

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
#from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [51]:
data = pd.read_parquet(r'D:\0_Respaldo\0_Proyectos_2024\ML_proyects\KagelX\KaggleX\booknotes\s1_t1\beta_data_03.parquet')
df = data.copy()
df.head(2)

,price,engine,accident,milage,model_year,milage_ratio,delta,Cluster
index,,,,,,,,
0,11000,719,1,74349,2018,36.842914,720,1
1,8250,534,1,80000,2007,39.860488,535,0


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 54034 entries, 0 to 54272
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         54034 non-null  int64  
 1   engine        54034 non-null  int32  
 2   accident      54034 non-null  int32  
 3   milage        54034 non-null  int64  
 4   model_year    54034 non-null  int64  
 5   milage_ratio  54034 non-null  float64
 6   delta         54034 non-null  int32  
 7   Cluster       54034 non-null  int8   
dtypes: float64(1), int32(3), int64(3), int8(1)
memory usage: 2.7 MB


In [52]:
x = df.drop(['price'], axis=1) #Features
y = df['price'] #target

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 28)

In [40]:
# from sklearn.model_selection import RandomizedSearchCV

In [42]:
# from scipy.stats import randint as sp_randint
# from scipy.stats import uniform as sp_uniform

In [64]:
# param_dist = {
#     'learning_rate': sp_uniform(loc=0.01, scale=0.09),
#     'num_leaves': sp_randint(6, 50),
#     'min_child_samples': sp_randint(100, 500),
#     'min_child_weight': [1e-5, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3, 1e4],
#     'subsample': sp_uniform(loc=0.2, scale=0.8),
#     'colsample_bytree': sp_uniform(loc=0.4, scale=0.6),
#     'reg_alpha': [0, 1e-1, 1, 2, 5, 7, 10, 50, 100],
#     'reg_lambda': [0, 1e-1, 1, 5, 10, 20, 50, 100]
# }

In [71]:
model = lgb.LGBMRegressor(objective='regression')

In [65]:
random_search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_dist,
    n_iter=100,
    scoring='neg_mean_squared_error',
    cv=3,
    verbose=1,
    random_state=42,
    n_jobs=-1
)

In [66]:
random_search.fit(x, y)

Fitting 3 folds for each of 100 candidates, totalling 300 fits
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001181 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1056
[LightGBM] [Info] Number of data points in the train set: 54034, number of used features: 7
[LightGBM] [Info] Start training from score 37217.862883


RandomizedSearchCV(cv=3,
                   estimator=LGBMRegressor(colsample_bytree=0.5218367348408617,
                                           min_child_samples=112,
                                           min_child_weight=100.0,
                                           num_leaves=27,
                                           objective='regression',
                                           reg_alpha=0.1, reg_lambda=0.1,
                                           subsample=0.8464963036515336),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'colsample_bytree': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0...
                                                             0.1, 1, 10.0,
                                                             100.0, 1000.0,
                                                             10000.0],
                                        'num_leaves': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001B6FDB46210>,
                                        'reg_alpha': [0, 0.1, 1, 2, 5, 7, 10,
                                                      50, 100],
                                        'reg_lambda': [0, 0.1, 1, 5, 10, 20, 50,
                                                       100],
                                        'subsample': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x000001B6FDB47010>},
                   random_state=42, scoring='neg_mean_squared_error',
                   verbose=1)

In [67]:
print("Mejores hiperparámetros encontrados:")
print(random_search.best_params_)

Mejores hiperparámetros encontrados:
{'colsample_bytree': 0.7836168141845454, 'learning_rate': 0.08245402007674256, 'min_child_samples': 202, 'min_child_weight': 10.0, 'num_leaves': 39, 'reg_alpha': 7, 'reg_lambda': 0.1, 'subsample': 0.6719341494837066}


In [68]:
# Modelo con los parametros optimizados
# model = lgb.LGBMRegressor(objective='regression',
#                           colsample_bytree = 0.5218367348408617,
#                           learning_rate = 0.08245402007674256,
#                           min_child_samples = 112,
#                           min_child_weight = 100.0,
#                           num_leaves = 27,
#                           reg_alpha = 0.1,
#                           reg_lambda = 0.1,
#                           subsample = 0.8464963036515336 
#                           )

In [72]:

model.fit(
    x_train, y_train,
    eval_set=[(x_test, y_test)],
    eval_metric='rmse',    
)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001064 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1056
[LightGBM] [Info] Number of data points in the train set: 43227, number of used features: 7
[LightGBM] [Info] Start training from score 37204.469406


LGBMRegressor(objective='regression')

In [73]:
y_pred = model.predict(x_test)
predictions = model.predict(x_test)

rmse = mean_squared_error(y_test, y_pred, squared=False)
test_mae = mean_absolute_error(y_test, predictions)

print(f'{test_mae=:.4f}')
print(f'RMSE: {rmse=:.4f}')

test_mae=14146.4663
RMSE: rmse=32605.3891


d:\0_Respaldo\0_Proyectos_2024\ML_proyects\KagelX\KaggleX\.venv\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [9]:
import joblib as jb

In [11]:
jb.dump(model, r'D:\0_Respaldo\0_Proyectos_2024\ML_proyects\KagelX\KaggleX\booknotes\s1_t1\model.pkl')

['D:\\0_Respaldo\\0_Proyectos_2024\\ML_proyects\\KagelX\\KaggleX\\booknotes\\s1_t1\\model.pkl']